# Intraday Tech Alpha — Minute → Hourly with News & Macro
Downloads **1‑minute bars** (Polygon) for selected tickers, resamples to **hourly**,
computes a few **technical indicators**, merges **hourly news sentiment counts**, and joins **daily macro**
(forward‑filled to hourly). Outputs both **CSV** and **Parquet**.

## Setup

In [ ]:
import os, time, json, datetime as dt
import pandas as pd
import numpy as np
import requests

from dotenv import load_dotenv
load_dotenv()

POLYGON_API_KEY = os.getenv("POLYGON_API_KEY")
FINNHUB_API_KEY = os.getenv("FINNHUB_API_KEY")
FRED_API_KEY    = os.getenv("FRED_API_KEY")

print("Keys present:", {
    "POLYGON": bool(POLYGON_API_KEY),
    "FINNHUB": bool(FINNHUB_API_KEY),
    "FRED":    bool(FRED_API_KEY)
})

OUT_DIR = "intraday_data"
os.makedirs(OUT_DIR, exist_ok=True)

# Universe & window
TICKERS = ["AAPL", "MSFT", "GOOGL", "NVDA", "META", "AMZN"]
DAYS_BACK = 5                 # past N trading days (ish)
END_TS = dt.datetime.now(dt.timezone.utc)
START_TS = END_TS - dt.timedelta(days=DAYS_BACK)

# Resample rule
RESAMPLE_RULE = "1H"          # hourly bars from minute data
PRINT_PREVIEW = True


Keys present: {'POLYGON': True, 'FINNHUB': True, 'FRED': True}


## Helpers

In [2]:
POLY_BASE    = "https://api.polygon.io"
FINNHUB_BASE = "https://finnhub.io/api/v1"
FRED_BASE    = "https://api.stlouisfed.org/fred/series/observations"

def _get_json_safe(url: str, params: dict, max_retries=3, backoff=1.25):
    last_err = None
    for i in range(max_retries):
        try:
            r = requests.get(url, params=params, timeout=30)
            if r.status_code == 429:
                time.sleep(backoff * (i+1))
                continue
            r.raise_for_status()
            return r.json()
        except Exception as e:
            last_err = e
            time.sleep(backoff * (i+1))
    print(f"[warn] GET failed: {url} | {type(last_err).__name__}: {last_err}")
    return None

def ensure_dtindex(df, col="time"):
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], utc=True, errors="coerce")
        df = df.dropna(subset=[col]).set_index(df[col]).sort_index()
        df = df.drop(columns=[col], errors="ignore")
    df.index = df.index.tz_convert("UTC")
    return df

def resample_ohlcv(df: pd.DataFrame, rule="1H"):
    agg = {"open": "first", "high": "max", "low": "min", "close":"last", "volume":"sum"}
    return df.resample(rule).agg(agg).dropna(subset=["open","high","low","close"])

def rsi(close: pd.Series, period: int = 14) -> pd.Series:
    delta = close.diff()
    up = delta.clip(lower=0).ewm(alpha=1/period, adjust=False).mean()
    down = (-delta.clip(upper=0)).ewm(alpha=1/period, adjust=False).mean()
    rs = up / down.replace(0, np.nan)
    return 100 - (100 / (1 + rs))

def ema(series: pd.Series, span: int) -> pd.Series:
    return series.ewm(span=span, adjust=False).mean()

def macd(close: pd.Series, fast=12, slow=26, signal=9):
    macd_line = ema(close, fast) - ema(close, slow)
    signal_line = ema(macd_line, signal)
    hist = macd_line - signal_line
    return macd_line, signal_line, hist

def vwap(df: pd.DataFrame, window: int = 30):
    pv = (df["close"] * df["volume"]).rolling(window, min_periods=1).sum()
    vol = df["volume"].rolling(window, min_periods=1).sum()
    return pv / vol.replace(0, np.nan)


## Download 1‑minute bars (Polygon primary, Finnhub fallback)

In [3]:
def polygon_minute_bars(ticker: str, start_ts: dt.datetime, end_ts: dt.datetime, timespan="minute", mult=1, limit=50000):
    if not POLYGON_API_KEY:
        return pd.DataFrame(columns=["time","open","high","low","close","volume"])
    url = f"{POLY_BASE}/v2/aggs/ticker/{ticker}/range/{mult}/{timespan}/{start_ts.isoformat()}/{end_ts.isoformat()}"
    params = {"adjusted":"true", "sort":"asc", "limit": limit, "apiKey": POLYGON_API_KEY}
    js = _get_json_safe(url, params)
    if not js or "results" not in js:
        print(f"[warn] Polygon empty for {ticker}")
        return pd.DataFrame(columns=["time","open","high","low","close","volume"])
    df = pd.DataFrame(js["results"])
    if df.empty:
        return pd.DataFrame(columns=["time","open","high","low","close","volume"])
    df.rename(columns={"t":"time","o":"open","h":"high","l":"low","c":"close","v":"volume"}, inplace=True)
    df["time"] = pd.to_datetime(df["time"], unit="ms", utc=True)
    return df[["time","open","high","low","close","volume"]]

def finnhub_minute_bars(ticker: str, start_ts: dt.datetime, end_ts: dt.datetime, resolution="1"):
    if not FINNHUB_API_KEY:
        return pd.DataFrame(columns=["time","open","high","low","close","volume"])
    url = f"{FINNHUB_BASE}/stock/candle"
    params = {"symbol": ticker, "resolution": resolution, "from": int(start_ts.timestamp()), "to": int(end_ts.timestamp()), "token": FINNHUB_API_KEY}
    js = _get_json_safe(url, params)
    if not js or js.get("s") != "ok":
        print(f"[warn] Finnhub empty for {ticker}")
        return pd.DataFrame(columns=["time","open","high","low","close","volume"])
    df = pd.DataFrame({
        "time": pd.to_datetime(js["t"], unit="s", utc=True),
        "open": js["o"], "high": js["h"], "low": js["l"], "close": js["c"], "volume": js["v"]
    })
    return df

def get_minute_data(ticker: str, start_ts: dt.datetime, end_ts: dt.datetime) -> pd.DataFrame:
    df = polygon_minute_bars(ticker, start_ts, end_ts)
    if df.empty:
        print(f"[info] Falling back to Finnhub minute for {ticker}")
        df = finnhub_minute_bars(ticker, start_ts, end_ts, resolution="1")
    df = ensure_dtindex(df, "time")
    return df

minute_data = {}
for t in TICKERS:
    print("Fetching minute bars for", t)
    dfm = get_minute_data(t, START_TS, END_TS)
    minute_data[t] = dfm
    if PRINT_PREVIEW and not dfm.empty:
        print(t, dfm.head(3), dfm.tail(3), sep="\n---\n")


Fetching minute bars for AAPL
[warn] GET failed: https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/minute/2025-09-30T23:59:08.299766+00:00/2025-10-05T23:59:08.299766+00:00 | HTTPError: 400 Client Error: Bad Request for url: https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/minute/2025-09-30T23:59:08.299766+00:00/2025-10-05T23:59:08.299766+00:00?adjusted=true&sort=asc&limit=50000&apiKey=TNRyzBuL5CNEIxE3aGKd66ssjVJkVq8B
[warn] Polygon empty for AAPL
[info] Falling back to Finnhub minute for AAPL
[warn] GET failed: https://finnhub.io/api/v1/stock/candle | HTTPError: 403 Client Error: Forbidden for url: https://finnhub.io/api/v1/stock/candle?symbol=AAPL&resolution=1&from=1759276748&to=1759708748&token=d39ir8hr01qoho9g1vb0d39ir8hr01qoho9g1vbg
[warn] Finnhub empty for AAPL
Fetching minute bars for MSFT
[warn] GET failed: https://api.polygon.io/v2/aggs/ticker/MSFT/range/1/minute/2025-09-30T23:59:08.299766+00:00/2025-10-05T23:59:08.299766+00:00 | HTTPError: 400 Client Error: Bad Request for 

## Resample to hourly and compute indicators

In [4]:
hourly_data = {}
for t, dfm in minute_data.items():
    if dfm.empty:
        hourly_data[t] = pd.DataFrame(columns=["open","high","low","close","volume"])
        continue

    dfm_feat = dfm.copy()
    dfm_feat["rsi_14"] = rsi(dfm_feat["close"], 14)
    macd_line, signal_line, hist = macd(dfm_feat["close"], 12, 26, 9)
    dfm_feat["macd"] = macd_line
    dfm_feat["macd_signal"] = signal_line
    dfm_feat["macd_hist"] = hist
    dfm_feat["vwap_30m"] = vwap(dfm_feat, 30)

    hr = resample_ohlcv(dfm_feat[["open","high","low","close","volume"]], RESAMPLE_RULE)
    ind = dfm_feat[["rsi_14","macd","macd_signal","macd_hist","vwap_30m"]].resample(RESAMPLE_RULE).last()
    hr = hr.join(ind, how="left")
    hourly_data[t] = hr

    out_csv = os.path.join(OUT_DIR, f"{t}_hourly.csv")
    out_parq = os.path.join(OUT_DIR, f"{t}_hourly.parquet")
    hr.to_csv(out_csv)
    try:
        hr.to_parquet(out_parq, index=True)
    except Exception as e:
        print(f"[warn] to_parquet failed for {t}: {e}")

    if PRINT_PREVIEW:
        print(f"[{t}] hourly shape:", hr.shape)


## Hourly news sentiment counts (Polygon or Finnhub)

In [16]:
try:
    import nltk
    from nltk.sentiment import SentimentIntensityAnalyzer
    try:
        _ = SentimentIntensityAnalyzer()
    except Exception:
        nltk.download('vader_lexicon')
        _ = SentimentIntensityAnalyzer()
    VADER_OK = True
except Exception:
    VADER_OK = False

def simple_sent(text: str) -> float:
    if not isinstance(text, str) or not text.strip(): return 0.0
    if VADER_OK:
        sia = SentimentIntensityAnalyzer()
        return float(sia.polarity_scores(text)["compound"])
    t = text.lower()
    pos = sum(w in t for w in ["beat","record","growth","surge","profit","upgrade","outperform","strong","rally"])
    neg = sum(w in t for w in ["miss","cut","probe","lawsuit","downgrade","decline","headwind","weak","plunge"])
    return (pos - neg) / 6.0

def polygon_news_window(ticker: str, start_ts: dt.datetime, end_ts: dt.datetime, limit=1000):
    if not POLYGON_API_KEY:
        return pd.DataFrame(columns=["time","headline","source"])
    url = f"{POLY_BASE}/v2/reference/news"
    params = {"ticker": ticker, "limit": min(limit,1000), "order": "desc",
              "published_utc.gte": start_ts.date().isoformat(),
              "published_utc.lte": end_ts.date().isoformat(),
              "apiKey": POLYGON_API_KEY}
    js = _get_json_safe(url, params)
    if not js or "results" not in js or not js["results"]:
        return pd.DataFrame(columns=["time","headline","source"])
    df = pd.DataFrame(js["results"])
    df["time"] = pd.to_datetime(df["published_utc"], utc=True, errors="coerce")
    df["headline"] = df.get("title","").fillna("")
    df["source"] = "polygon"
    return df[["time","headline","source"]].dropna(subset=["time"]).sort_values("time")

def finnhub_news_window(ticker: str, start_ts: dt.datetime, end_ts: dt.datetime):
    if not FINNHUB_API_KEY:
        return pd.DataFrame(columns=["time","headline","source"])
    url = f"{FINNHUB_BASE}/company-news"
    params = {"symbol": ticker, "from": start_ts.date().isoformat(), "to": end_ts.date().isoformat(), "token": FINNHUB_API_KEY}
    js = _get_json_safe(url, params)
    if not isinstance(js, list) or not js:
        return pd.DataFrame(columns=["time","headline","source"])
    df = pd.DataFrame(js)
    if "datetime" in df.columns:
        df["time"] = pd.to_datetime(df["datetime"], unit="s", utc=True, errors="coerce")
    else:
        df["time"] = pd.to_datetime(df.get("time", pd.NaT), unit="ms", utc=True, errors="coerce")
    df["headline"] = df.get("headline","").fillna("")
    df["source"] = "finnhub"
    return df[["time","headline","source"]].dropna(subset=["time"]).sort_values("time")

def hourly_news_sentiment_counts(ticker: str, start_ts: dt.datetime, end_ts: dt.datetime) -> pd.DataFrame:
    poly = polygon_news_window(ticker, start_ts, end_ts)
    fin  = finnhub_news_window(ticker, start_ts, end_ts)
    news = pd.concat([poly, fin], ignore_index=True)
    if news.empty:
        return pd.DataFrame(columns=["pos_cnt","neg_cnt","tot_cnt","sent_mean"])
    news["sent"] = news["headline"].astype(str).apply(simple_sent)
    news = news.dropna(subset=["time"])
    news = news.set_index(pd.to_datetime(news["time"], utc=True)).sort_index()
    agg = pd.DataFrame({
        "pos_cnt": (news["sent"] >  0.2).resample(RESAMPLE_RULE).sum(),
        "neg_cnt": (news["sent"] < -0.2).resample(RESAMPLE_RULE).sum(),
        "tot_cnt": news["sent"].resample(RESAMPLE_RULE).size(),
        "sent_mean": news["sent"].resample(RESAMPLE_RULE).mean()
    })
    return agg

hourly_news = {}
for t in TICKERS:
    print("Building hourly news sentiment for", t)
    s = hourly_news_sentiment_counts(t, START_TS, END_TS)
    hourly_news[t] = s
    s.to_csv(os.path.join(OUT_DIR, f"{t}_hourly_news.csv"))
    if PRINT_PREVIEW and not s.empty:
        print(t, s.tail(3), sep="\n")


Building hourly news sentiment for AAPL


C:\Users\SyedM\AppData\Local\Temp\ipykernel_32252\899349796.py:67: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  "pos_cnt": (news["sent"] >  0.2).resample(RESAMPLE_RULE).sum(),
C:\Users\SyedM\AppData\Local\Temp\ipykernel_32252\899349796.py:68: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  "neg_cnt": (news["sent"] < -0.2).resample(RESAMPLE_RULE).sum(),
C:\Users\SyedM\AppData\Local\Temp\ipykernel_32252\899349796.py:69: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  "tot_cnt": news["sent"].resample(RESAMPLE_RULE).size(),
C:\Users\SyedM\AppData\Local\Temp\ipykernel_32252\899349796.py:70: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  "sent_mean": news["sent"].resample(RESAMPLE_RULE).mean()


AAPL
                           pos_cnt  neg_cnt  tot_cnt  sent_mean
time                                                           
2025-10-05 21:00:00+00:00        0        0        0        NaN
2025-10-05 22:00:00+00:00        0        0        0        NaN
2025-10-05 23:00:00+00:00        1        0        1     0.3182
Building hourly news sentiment for MSFT


C:\Users\SyedM\AppData\Local\Temp\ipykernel_32252\899349796.py:67: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  "pos_cnt": (news["sent"] >  0.2).resample(RESAMPLE_RULE).sum(),
C:\Users\SyedM\AppData\Local\Temp\ipykernel_32252\899349796.py:68: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  "neg_cnt": (news["sent"] < -0.2).resample(RESAMPLE_RULE).sum(),
C:\Users\SyedM\AppData\Local\Temp\ipykernel_32252\899349796.py:69: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  "tot_cnt": news["sent"].resample(RESAMPLE_RULE).size(),
C:\Users\SyedM\AppData\Local\Temp\ipykernel_32252\899349796.py:70: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  "sent_mean": news["sent"].resample(RESAMPLE_RULE).mean()


MSFT
                           pos_cnt  neg_cnt  tot_cnt  sent_mean
time                                                           
2025-10-05 21:00:00+00:00        1        0        1     0.3818
2025-10-05 22:00:00+00:00        0        0        0        NaN
2025-10-05 23:00:00+00:00        1        0        1     0.3182
Building hourly news sentiment for GOOGL


...

C:\Users\SyedM\AppData\Local\Temp\ipykernel_32252\899349796.py:67: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  "pos_cnt": (news["sent"] >  0.2).resample(RESAMPLE_RULE).sum(),
C:\Users\SyedM\AppData\Local\Temp\ipykernel_32252\899349796.py:68: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  "neg_cnt": (news["sent"] < -0.2).resample(RESAMPLE_RULE).sum(),
C:\Users\SyedM\AppData\Local\Temp\ipykernel_32252\899349796.py:69: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  "tot_cnt": news["sent"].resample(RESAMPLE_RULE).size(),
C:\Users\SyedM\AppData\Local\Temp\ipykernel_32252\899349796.py:70: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  "sent_mean": news["sent"].resample(RESAMPLE_RULE).mean()


NVDA
                           pos_cnt  neg_cnt  tot_cnt  sent_mean
time                                                           
2025-10-05 21:00:00+00:00        1        0        1     0.3818
2025-10-05 22:00:00+00:00        0        0        1     0.0000
2025-10-05 23:00:00+00:00        1        0        1     0.3182
Building hourly news sentiment for META


C:\Users\SyedM\AppData\Local\Temp\ipykernel_32252\899349796.py:67: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  "pos_cnt": (news["sent"] >  0.2).resample(RESAMPLE_RULE).sum(),
C:\Users\SyedM\AppData\Local\Temp\ipykernel_32252\899349796.py:68: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  "neg_cnt": (news["sent"] < -0.2).resample(RESAMPLE_RULE).sum(),
C:\Users\SyedM\AppData\Local\Temp\ipykernel_32252\899349796.py:69: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  "tot_cnt": news["sent"].resample(RESAMPLE_RULE).size(),
C:\Users\SyedM\AppData\Local\Temp\ipykernel_32252\899349796.py:70: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  "sent_mean": news["sent"].resample(RESAMPLE_RULE).mean()


META
                           pos_cnt  neg_cnt  tot_cnt  sent_mean
time                                                           
2025-10-05 21:00:00+00:00        0        0        0        NaN
2025-10-05 22:00:00+00:00        0        0        0        NaN
2025-10-05 23:00:00+00:00        1        0        1     0.3182
Building hourly news sentiment for AMZN
AMZN
                           pos_cnt  neg_cnt  tot_cnt  sent_mean
time                                                           
2025-10-05 21:00:00+00:00        0        0        0        NaN
2025-10-05 22:00:00+00:00        0        0        0        NaN
2025-10-05 23:00:00+00:00        1        0        1     0.3182


C:\Users\SyedM\AppData\Local\Temp\ipykernel_32252\899349796.py:67: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  "pos_cnt": (news["sent"] >  0.2).resample(RESAMPLE_RULE).sum(),
C:\Users\SyedM\AppData\Local\Temp\ipykernel_32252\899349796.py:68: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  "neg_cnt": (news["sent"] < -0.2).resample(RESAMPLE_RULE).sum(),
C:\Users\SyedM\AppData\Local\Temp\ipykernel_32252\899349796.py:69: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  "tot_cnt": news["sent"].resample(RESAMPLE_RULE).size(),
C:\Users\SyedM\AppData\Local\Temp\ipykernel_32252\899349796.py:70: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  "sent_mean": news["sent"].resample(RESAMPLE_RULE).mean()


## Daily macro (FRED) forward‑filled to hourly

In [6]:
def fred_series_daily(series_id: str, start: str, end: str) -> pd.Series:
    if not FRED_API_KEY:
        return pd.Series(dtype=float)
    url = FRED_BASE
    params = {"series_id": series_id, "api_key": FRED_API_KEY, "file_type":"json",
              "frequency":"d", "observation_start": start, "observation_end": end}
    js = _get_json_safe(url, params)
    obs = (js or {}).get("observations", [])
    if not obs:
        return pd.Series(dtype=float)
    df = pd.DataFrame(obs)
    df["date"] = pd.to_datetime(df["date"], utc=True, errors="coerce")
    df["value"] = pd.to_numeric(df["value"], errors="coerce")
    return df.set_index("date")["value"].astype(float)

start_str = START_TS.date().isoformat()
end_str   = END_TS.date().isoformat()

dgs10  = fred_series_daily("DGS10", start_str, end_str)
dff    = fred_series_daily("DFF",   start_str, end_str)
vix    = fred_series_daily("VIXCLS",start_str, end_str)

macro_daily = pd.DataFrame({"dgs10": dgs10, "dff": dff, "vix": vix}).dropna(how="all")
macro_daily.index = macro_daily.index.tz_convert("UTC")
macro_daily.to_csv(os.path.join(OUT_DIR, "macro_daily.csv"))
print("Macro daily head:\n", macro_daily.head())


Macro daily head:
                            dgs10   dff    vix
date                                         
2025-09-30 00:00:00+00:00   4.16  4.09  16.28
2025-10-01 00:00:00+00:00   4.12  4.09  16.29
2025-10-02 00:00:00+00:00   4.10  4.09  16.63


## Merge hourly bars, hourly news, and daily macro (ffill)

In [7]:
merged_hourly = {}

for t, h in hourly_data.items():
    if h.empty:
        merged_hourly[t] = pd.DataFrame()
        continue
    hn = hourly_news.get(t, pd.DataFrame())
    # Build hourly index covering the data
    idx = h.index
    # Align macro to this index and ffill
    macro_h = macro_daily.reindex(pd.DatetimeIndex(idx)).ffill()

    m = h.join(hn, how="left").join(macro_h, how="left")
    merged_hourly[t] = m

    out_csv = os.path.join(OUT_DIR, f"{t}_hourly_merged.csv")
    out_parq = os.path.join(OUT_DIR, f"{t}_hourly_merged.parquet")
    m.to_csv(out_csv)
    try:
        m.to_parquet(out_parq, index=True)
    except Exception as e:
        print(f"[warn] to_parquet failed for merged {t}: {e}")

    print(t, "merged hourly shape:", m.shape)
    if PRINT_PREVIEW:
        print(m.tail(2))


## Build panel (MultiIndex columns by ticker)

In [8]:
panel = pd.concat(merged_hourly, axis=1) if merged_hourly else pd.DataFrame()
panel_csv = os.path.join(OUT_DIR, "panel_hourly.csv")
panel_parq = os.path.join(OUT_DIR, "panel_hourly.parquet")
panel.to_csv(panel_csv)
try:
    panel.to_parquet(panel_parq, index=True)
except Exception as e:
    print(f"[warn] to_parquet failed for panel: {e}")
print("Panel shape:", panel.shape)
panel.tail(3)


Panel shape: (0, 0)


Empty DataFrame
Columns: []
Index: []

# Yahoo Finance minute by minute call

In [11]:
import yfinance as yf
import pandas as pd
import datetime as dt

from dotenv import load_dotenv
load_dotenv()

POLYGON_API_KEY = os.getenv("POLYGON_API_KEY")
FINNHUB_API_KEY = os.getenv("FINNHUB_API_KEY")
FRED_API_KEY    = os.getenv("FRED_API_KEY")

print("Keys present:", {
    "POLYGON": bool(POLYGON_API_KEY),
    "FINNHUB": bool(FINNHUB_API_KEY),
    "FRED":    bool(FRED_API_KEY)
})

OUT_DIR = "../data_cache/Intraday"
os.makedirs(OUT_DIR, exist_ok=True)

# --- Parameters ---
TICKERS = ["AAPL", "MSFT", "GOOGL", "NVDA", "META", "AMZN"]
DAYS_BACK = 7                 # past N trading days (ish)

START  = (dt.datetime.utcnow() - dt.timedelta(days=DAYS_BACK)).strftime("%Y-%m-%d")
END    = dt.datetime.utcnow().strftime("%Y-%m-%d")


Keys present: {'POLYGON': True, 'FINNHUB': True, 'FRED': True}


C:\Users\SyedM\AppData\Local\Temp\ipykernel_32252\2653464459.py:25: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  START  = (dt.datetime.utcnow() - dt.timedelta(days=DAYS_BACK)).strftime("%Y-%m-%d")
C:\Users\SyedM\AppData\Local\Temp\ipykernel_32252\2653464459.py:26: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  END    = dt.datetime.utcnow().strftime("%Y-%m-%d")


## Yahoo News Dataset training

In [12]:

for TICKER in TICKERS:
    # --- Fetch minute or hourly data ---
    # interval options: "1m", "5m", "15m", "30m", "60m", "90m", "1h"
    data_minute = yf.download(TICKER, start=START, end=END, interval="1m", progress=False)
    data_hourly = yf.download(TICKER, start=START, end=END, interval="1h", progress=False)

    # --- Clean and display ---
    print("Minute data sample:")
    display(data_minute.head())

    print("\nHourly data sample:")
    display(data_hourly.head())

    # --- Save to CSV if desired ---
    data_minute.to_csv(os.path.join(OUT_DIR, f"{TICKER}_minute.csv"))
    data_hourly.to_csv(os.path.join(OUT_DIR,f"{TICKER}_hourly.csv"))

    # --- Compute returns and simple features ---
    data_hourly["ret"] = data_hourly["Close"].pct_change()
    data_hourly["volatility"] = data_hourly["ret"].rolling(20).std()
    data_hourly["sma20"] = data_hourly["Close"].rolling(20).mean()
    data_hourly["sma50"] = data_hourly["Close"].rolling(50).mean()

C:\Users\SyedM\AppData\Local\Temp\ipykernel_32252\2289027016.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data_minute = yf.download(TICKER, start=START, end=END, interval="1m", progress=False)


Minute data sample:


C:\Users\SyedM\AppData\Local\Temp\ipykernel_32252\2289027016.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data_hourly = yf.download(TICKER, start=START, end=END, interval="1h", progress=False)


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Datetime,,,,,
2025-09-29 13:30:00+00:00,254.050003,254.274994,253.910004,254.000000,1272275
2025-09-29 13:31:00+00:00,253.675003,254.389999,253.520004,254.110001,210664
2025-09-29 13:32:00+00:00,254.300003,254.300003,253.565002,253.699997,186692
2025-09-29 13:33:00+00:00,254.119995,254.320007,253.880005,254.240005,98311
2025-09-29 13:34:00+00:00,253.639893,254.149902,253.500000,254.119995,140683



Hourly data sample:


...

C:\Users\SyedM\AppData\Local\Temp\ipykernel_32252\2289027016.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data_minute = yf.download(TICKER, start=START, end=END, interval="1m", progress=False)
C:\Users\SyedM\AppData\Local\Temp\ipykernel_32252\2289027016.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data_hourly = yf.download(TICKER, start=START, end=END, interval="1h", progress=False)


Minute data sample:


Price,Close,High,Low,Open,Volume
Ticker,AMZN,AMZN,AMZN,AMZN,AMZN
Datetime,,,,,
2025-09-29 13:30:00+00:00,220.250000,220.320007,219.830002,220.000000,1611295
2025-09-29 13:31:00+00:00,220.070007,220.439102,219.759995,220.059998,217154
2025-09-29 13:32:00+00:00,220.331100,220.389999,219.710098,220.039993,158951
2025-09-29 13:33:00+00:00,220.138504,220.369995,220.039993,220.315002,111552
2025-09-29 13:34:00+00:00,220.070007,220.199997,220.009995,220.125000,76408



Hourly data sample:


Price,Close,High,Low,Open,Volume
Ticker,AMZN,AMZN,AMZN,AMZN,AMZN
Datetime,,,,,
2025-09-29 13:30:00+00:00,221.430099,222.089996,219.710098,220.000000,6887387
2025-09-29 14:30:00+00:00,220.676300,221.538803,220.550003,221.449997,2847139
2025-09-29 15:30:00+00:00,221.361298,221.600296,220.580093,220.710007,1914981
2025-09-29 16:30:00+00:00,221.770004,221.830002,220.610001,221.350006,2992178
2025-09-29 17:30:00+00:00,221.514999,221.800003,220.919998,221.779999,1824169
